In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

In [2]:
train_data = pd.read_csv(r"D:\FlightRacePrediction\notebooks\data\Data_Train.csv")

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


In [5]:
train_data["Journey_day"] = pd.to_datetime(train_data.Date_of_Journey, format="%d/%m/%Y").dt.day

In [6]:
train_data["Journey_month"] = pd.to_datetime(train_data["Date_of_Journey"], format = "%d/%m/%Y").dt.month

In [7]:
train_data.drop(["Date_of_Journey"], axis = 1, inplace = True)

In [8]:
# Extracting Hours
train_data["Dep_hour"] = pd.to_datetime(train_data["Dep_Time"]).dt.hour

# Extracting Minutes
train_data["Dep_min"] = pd.to_datetime(train_data["Dep_Time"]).dt.minute

# Now we can drop Dep_Time as it is of no use
train_data.drop(["Dep_Time"], axis = 1, inplace = True)

C:\Users\tejav\AppData\Local\Temp\ipykernel_21956\2961852048.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  train_data["Dep_hour"] = pd.to_datetime(train_data["Dep_Time"]).dt.hour
C:\Users\tejav\AppData\Local\Temp\ipykernel_21956\2961852048.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  train_data["Dep_min"] = pd.to_datetime(train_data["Dep_Time"]).dt.minute


In [9]:
# Extracting Hours
train_data["Arrival_hour"] = pd.to_datetime(train_data.Arrival_Time).dt.hour

# Extracting Minutes
train_data["Arrival_min"] = pd.to_datetime(train_data.Arrival_Time).dt.minute

# Now we can drop Arrival_Time as it is of no use
train_data.drop(["Arrival_Time"], axis = 1, inplace = True)

C:\Users\tejav\AppData\Local\Temp\ipykernel_21956\1276782324.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  train_data["Arrival_hour"] = pd.to_datetime(train_data.Arrival_Time).dt.hour
C:\Users\tejav\AppData\Local\Temp\ipykernel_21956\1276782324.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  train_data["Arrival_min"] = pd.to_datetime(train_data.Arrival_Time).dt.minute


In [10]:
duration = list(train_data["Duration"])

for i in range(len(duration)):
    if len(duration[i].split()) != 2:    # Check if duration contains only hour or mins
        if "h" in duration[i]:
            duration[i] = duration[i].strip() + " 0m"   # Adds 0 minute
        else:
            duration[i] = "0h " + duration[i]           # Adds 0 hour

duration_hours = []
duration_mins = []
for i in range(len(duration)):
    duration_hours.append(int(duration[i].split(sep = "h")[0]))    # Extract hours from duration
    duration_mins.append(int(duration[i].split(sep = "m")[0].split()[-1]))  

In [11]:
train_data["Duration_hours"] = duration_hours
train_data["Duration_mins"] = duration_mins

In [12]:
train_data.drop(["Duration"], axis = 1, inplace = True)

In [13]:
train_data.drop(["Additional_Info"], axis = 1, inplace = True)

In [14]:
# Additional_Info contains almost 80% no_info
# Route and Total_Stops are related to each other

train_data.drop(["Route"], axis = 1, inplace = True)

In [15]:
train_data.head()

,Airline,Source,Destination,Total_Stops,Price,Journey_day,Journey_month,Dep_hour,Dep_min,Arrival_hour,Arrival_min,Duration_hours,Duration_mins
0,IndiGo,Banglore,New Delhi,non-stop,3897,24,3,22,20,1,10,2,50
1,Air India,Kolkata,Banglore,2 stops,7662,1,5,5,50,13,15,7,25
2,Jet Airways,Delhi,Cochin,2 stops,13882,9,6,9,25,4,25,19,0
3,IndiGo,Kolkata,Banglore,1 stop,6218,12,5,18,5,23,30,5,25
4,IndiGo,Banglore,New Delhi,1 stop,13302,1,3,16,50,21,35,4,45


In [16]:
train_data.replace({"non-stop": 0, "1 stop": 1, "2 stops": 2, "3 stops": 3, "4 stops": 4}, inplace = True)

In [17]:
train_data.shape

(10683, 13)

In [18]:
# Segregating numerical and categorical variables
categorical_cols = train_data.select_dtypes(include='object').columns
numerical_cols = train_data.select_dtypes(exclude='object').columns

In [19]:
categorical_cols

Index(['Airline', 'Source', 'Destination'], dtype='object')

In [20]:
numerical_cols

Index(['Total_Stops', 'Price', 'Journey_day', 'Journey_month', 'Dep_hour',
       'Dep_min', 'Arrival_hour', 'Arrival_min', 'Duration_hours',
       'Duration_mins'],
      dtype='object')

In [24]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

In [25]:
pip install xgboost


Note: you may need to restart the kernel to use updated packages.


In [26]:
# Identify index of 'Price' column
price_index = train_data.columns.get_loc('Price')

# Rearrange columns so that 'Price' is at the end
cols = list(train_data.columns)
cols = cols[:price_index] + cols[price_index+1:] + [cols[price_index]]

In [27]:
df=train_data[cols]

In [28]:
df.head()

,Airline,Source,Destination,Total_Stops,Journey_day,Journey_month,Dep_hour,Dep_min,Arrival_hour,Arrival_min,Duration_hours,Duration_mins,Price
0,IndiGo,Banglore,New Delhi,0.0,24,3,22,20,1,10,2,50,3897
1,Air India,Kolkata,Banglore,2.0,1,5,5,50,13,15,7,25,7662
2,Jet Airways,Delhi,Cochin,2.0,9,6,9,25,4,25,19,0,13882
3,IndiGo,Kolkata,Banglore,1.0,12,5,18,5,23,30,5,25,6218
4,IndiGo,Banglore,New Delhi,1.0,1,3,16,50,21,35,4,45,13302


In [29]:
# Apply preprocessing to train_data
X = df.drop('Price', axis=1)
y = df['Price']

In [30]:
df.dtypes

Airline            object
Source             object
Destination        object
Total_Stops       float64
Journey_day         int32
Journey_month       int32
Dep_hour            int32
Dep_min             int32
Arrival_hour        int32
Arrival_min         int32
Duration_hours      int64
Duration_mins       int64
Price               int64
dtype: object

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [32]:
# Segregating numerical and categorical variables
categorical_cols = ['Airline', 'Source', 'Destination', 'Total_Stops']
numerical_cols = ['Journey_day', 'Journey_month', 'Dep_hour', 'Dep_min',
                  'Arrival_hour', 'Arrival_min', 'Duration_hours', 'Duration_mins']

# Preprocessing pipelines
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(drop='first', sparse=False))
])

# Apply ColumnTransformer to preprocess numerical and categorical columns
preprocessor = ColumnTransformer([
    ('num', num_pipeline, numerical_cols),
    ('cat', cat_pipeline, categorical_cols)
])

# Separate features (X) and target (y)
X = train_data.drop('Price', axis=1)
y = train_data['Price']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit and transform data using the preprocessor
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

# Define models and evaluation function
def train_and_evaluate_models(X_train, X_test, y_train, y_test):
    models = {
        'DecisionTreeRegressor': DecisionTreeRegressor(random_state=42),
        'RandomForestRegressor': RandomForestRegressor(random_state=42),
        'XGBRegressor': XGBRegressor(random_state=42)
    }

    model_list = []
    rmse_list = []
    r2_list = []

    for name, model in models.items():
        # Train the model
        model.fit(X_train, y_train)

        # Make predictions
        y_pred = model.predict(X_test)

        # Evaluate the model
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        r2 = model.score(X_test, y_test)

        # Store results
        model_list.append(name)
        rmse_list.append(rmse)
        r2_list.append(r2)

        # Print evaluation metrics
        print(f"Model: {name}")
        print("RMSE:", rmse)
        print("R2 Score:", r2)
        print("=" * 35)
        print()

    return model_list, rmse_list, r2_list

# Call the function with preprocessed data
model_list, rmse_list, r2_list = train_and_evaluate_models(X_train_processed, X_test_processed, y_train, y_test)

d:\Flight_Price_Prediction\venv\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Model: DecisionTreeRegressor
RMSE: 2737.645701866555
R2 Score: 0.6458743144252508

Model: RandomForestRegressor
RMSE: 2121.3622779045745
R2 Score: 0.7873660398869942

Model: XGBRegressor
RMSE: 1785.1153549695155
R2 Score: 0.8494310176946869



##### XGBRegressor works well